In [39]:
import openclean
import glob
import pandas as pd
import numpy as np


In [40]:
# https://data.cityofnewyork.us/Housing-Development/Housing-New-York-Units-by-Building/hg8x-zxpr
# https://data.cityofnewyork.us/Housing-Development/DOB-NOW-Build-Approved-Permits/rbx6-tga4
# https://data.cityofnewyork.us/Housing-Development/Buildings-Selected-for-the-Alternative-Enforcement/hcir-3275

# Data Downloading

Download the data using openClean

In [128]:
import gzip
import humanfriendly
import os

from openclean.data.source.socrata import Socrata

dataset = Socrata().dataset('rbx6-tga4')
datafile = './rbx6-tga4.tsv.gz'

if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)


fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))

Using 'DOB NOW: Build – Approved Permits' in file ./rbx6-tga4.tsv.gz of size 23.89 MB


# Data Loading

Load the data into pandas and openClean dataset object

In [42]:
import pandas as pd
from openclean.pipeline import stream

df  = pd.read_csv(datafile, dtype='object', sep='\t')
ds = stream(datafile)

In [43]:
np.__version__

'1.18.5'

In [44]:
pd.__version__

'1.2.4'

In [45]:
import glob

In [46]:
glob.glob("*")

['Buildings-Selected-for-the-Alternative-Enforcement-hcir-3275.ipynb',
 'DOB-NOW-Build-Approved-Permits-rbx6-tga4.ipynb',
 'hcir-3275.tsv.gz',
 'hcir-3275_cleaned_data.csv',
 'hg8x-zxpr.tsv.gz',
 'ic3t-wcy2.tsv.gz',
 'merged2.ipynb',
 'rbx6-tga4.tsv.gz',
 'Untitled.ipynb']

### Get some basic info about the dataset columns

In [47]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241445 entries, 0 to 241444
Data columns (total 35 columns):
 #   Column                                Non-Null Count   Dtype 
---  ------                                --------------   ----- 
 0   Job Filing Number                     241445 non-null  object
 1   Filing Reason                         241445 non-null  object
 2   House No                              241445 non-null  object
 3   Street Name                           241445 non-null  object
 4   Borough                               241445 non-null  object
 5   LOT                                   241297 non-null  object
 6   Bin                                   241445 non-null  object
 7   Block                                 241445 non-null  object
 8   C B NO                                241308 non-null  object
 9   Apt/Condo No(s)                       1645 non-null    object
 10  Work on Floor                         240701 non-null  object
 11  Work Type    

<ipython-input-47-6ac79f7ef903>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose=True, null_counts=True)


If any rows are complete duplicates, drop them

In [48]:
df = df.drop_duplicates()

Take an a look at some of the rows to get an idea of what the datset looks like

In [49]:
df

,Job Filing Number,Filing Reason,House No,Street Name,Borough,LOT,Bin,Block,C B NO,Apt/Condo No(s),...,Issued Date,Expired Date,Job Description,Estimated Job Costs,Owner Business Name,Owner Name,Owner Street Address,Owner City,Owner State,Owner Zip Code
0,M00531234-I1,Initial Permit,26,SHERMAN AVENUE,MANHATTAN,98,1064238,2175,112,NaN,...,06/23/2021 12:00:00 AM,06/23/2022 12:00:00 AM,Installation of a temporarily,5000,PR,SHLOIME GROSS,NaN,NaN,NaN,NaN
1,X00496654-I1,Initial Permit,1051,WESTCHESTER AVENUE,BRONX,64,2005808,2727,202,NaN,...,05/06/2021 12:00:00 AM,12/31/2021 12:00:00 AM,INSTALL EXTERIOR ILLUMINATED W,1850,EIB WESTCHESTER 1 LLC,FRANK SCAFFA,NaN,NaN,NaN,NaN
2,B00579209-I1,Initial Permit,1010,PACIFIC STREET,BROOKLYN,32,3428849,1133,308,NaN,...,08/19/2021 12:00:00 AM,07/20/2022 12:00:00 AM,INSTALLATION OF SCAFFOLD FOR W,1000,TWIN GROUP ASSOCIATES INC,SOLOMON SCHWIMMER,NaN,NaN,NaN,NaN
3,M00514656-I1,Initial Permit,33,WEST 125 STREET,MANHATTAN,21,1053501,1723,110,NaN,...,05/06/2021 12:00:00 AM,04/01/2022 12:00:00 AM,Installation of temporary side,1,"CGM MANAGEMENT GROUP, INC",CARMINE MOLISSE,NaN,NaN,NaN,NaN
4,M00431823-I1,Initial Permit,301,EAST 73 STREET,MANHATTAN,1,1044897,1448,108,NaN,...,06/04/2021 12:00:00 AM,03/11/2022 05:00:00 AM,Filing as per plans for the in,5000,BROADWAY MERCER ASSOCIATES,CYNTHIA GONZALEZ,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241440,B00269752-I1,Renewal Permit Without Changes,1,MAIN STREET,BROOKLYN,7501,3346104,27,302,NaN,...,07/10/2020 12:00:00 AM,02/23/2021 12:00:00 AM,PAA being filed to reflect cha,26000,NaN,MARY FRANCES SHAUGHNESSY,NaN,NaN,NaN,NaN
241441,B00239369-I1,Initial Permit,388,CHRISTOPHER AVENUE,BROOKLYN,128,3084828,3811,316,NaN,...,12/23/2020 12:00:00 AM,07/20/2021 12:00:00 AM,INSTALLATION OF SPRINKLER SYST,1000,PR,CHAIM SOBEL,NaN,NaN,NaN,NaN
241442,M00083385-I1,Renewal Permit Without Changes,25,MURRAY STREET,MANHATTAN,7505,1001451,134,101,NaN,...,01/21/2020 12:00:00 AM,11/03/2020 12:00:00 AM,INSTALLATION OF TEMPORARY (60',1000,PHOENIX SUTTON STR INC,PIOTR SIWIEC,NaN,NaN,NaN,NaN
241443,M00052214-I1,Initial Permit,1526,1 AVENUE,MANHATTAN,48,1050355,1559,108,NaN,...,07/06/2018 12:00:00 AM,09/30/2018 12:00:00 AM,installation of construction f,1000,NORDEST SERVICE LLC,JACK CALIENDO,NaN,NaN,NaN,NaN


## Describe columns in groups so they fit on screen

In [50]:
df[df.columns[:20]].describe()

,Job Filing Number,Filing Reason,House No,Street Name,Borough,LOT,Bin,Block,C B NO,Apt/Condo No(s),Work on Floor,Work Type,Permittee's License Type,Applicant License #,Applicant First Name,Applicant Middle Name,Applicant Last Name,Applicant Business Name,Applicant Business Address,Filing Representative First Name
count,241160,241160,241160,241160,241160,241012,241160,241160,241023,1642,240439,241160,241160,241160,241160,103,241159,240845,214540,134035
unique,154749,4,13571,5601,5,761,59990,10146,70,91,7464,17,8,9098,3394,1,6287,9407,9235,1527
top,M00013853-I1,Initial Permit,1,BROADWAY,MANHATTAN,1,1000855,16,105,OSP,Open Space,Sidewalk Shed,GC,032158,MICHAEL,GIL,SINGH,EVEREST SCAFFOLDING INC,49-30 31ST PLACE,ELY
freq,51,168465,2202,6896,104056,30378,322,577,25925,1188,32033,47166,165325,3949,7870,103,19677,4054,2727,12742


In [51]:
# Notes:
# Building Type looks binary and has 2 values + maybe NAN
# Cluster looks binary and has 2 values + maybe NAN
# Landmarked looks binary and has 4 values + maybe NAN
# Adult Establishment looks binary and has 2 values + maybe NAN
# Loft Board looks binary and has 2 values + maybe NAN
# City Owned looks binary and has 4 values + maybe NAN
# Little e looks binary and has 5 values + maybe NAN


In [52]:
df[df.columns[20:40]].describe()

,Filing Representative Middle Initial,Filing Representative Last Name,Filing Representative Business Name,Work Permit,Approved Date,Issued Date,Expired Date,Job Description,Estimated Job Costs,Owner Business Name,Owner Name,Owner Street Address,Owner City,Owner State,Owner Zip Code
count,0,134032,132705,241160,241130,241160,241129,241159,241159,226308,241156,0,0,0,0
unique,0,2039,2280,158655,5396,1254,2883,50530,17891,17520,32773,0,0,0,0
top,NaN,SEPULVEDA,AE DESIGN SOLUTION INC.,M00013853-I1-SH,01/21/2020 12:00:00 AM,11/15/2021 12:00:00 AM,08/06/2020 12:00:00 AM,INSTALLATION OF HEAVY DUTY SID,1000,PR,CHIRAG PATEL,NaN,NaN,NaN,NaN
freq,NaN,12808,12741,21,515,713,6213,8995,23230,42463,4396,NaN,NaN,NaN,NaN


In [53]:
df.columns

Index(['Job Filing Number', 'Filing Reason', 'House No', 'Street Name',
       'Borough', 'LOT', 'Bin', 'Block', 'C B NO', 'Apt/Condo No(s)',
       'Work on Floor', 'Work Type', 'Permittee's License Type',
       'Applicant License #', 'Applicant First Name', 'Applicant Middle Name',
       'Applicant Last Name', 'Applicant Business Name',
       'Applicant Business Address', 'Filing Representative First Name',
       'Filing Representative Middle Initial',
       'Filing Representative Last Name',
       'Filing Representative Business Name', 'Work Permit', 'Approved Date',
       'Issued Date', 'Expired Date', 'Job Description', 'Estimated Job Costs',
       'Owner Business Name', 'Owner Name', 'Owner Street Address',
       'Owner City', 'Owner State', 'Owner Zip Code'],
      dtype='object')

### Examining Applicant License #

Some repition in the Applicant License #, but nothing major. We will check some of the repeated Applicant License # to be sure they actually refer to the same jobs

In [54]:
df['Applicant License #'].value_counts(dropna=False)

032158    3949
607447    3503
610440    3321
617232    2374
606141    2361
          ... 
612835       1
621504       1
609965       1
605013       1
006834       1
Name: Applicant License #, Length: 9098, dtype: int64

Nothing weird looking here

In [55]:
df['Applicant License #'].min()

'000002'

In [56]:
df['Applicant License #'].max()

'999999'

In [57]:
df.loc[df['Applicant License #'].str.startswith('0')]

,Job Filing Number,Filing Reason,House No,Street Name,Borough,LOT,Bin,Block,C B NO,Apt/Condo No(s),...,Issued Date,Expired Date,Job Description,Estimated Job Costs,Owner Business Name,Owner Name,Owner Street Address,Owner City,Owner State,Owner Zip Code
1,X00496654-I1,Initial Permit,1051,WESTCHESTER AVENUE,BRONX,64,2005808,2727,202,NaN,...,05/06/2021 12:00:00 AM,12/31/2021 12:00:00 AM,INSTALL EXTERIOR ILLUMINATED W,1850,EIB WESTCHESTER 1 LLC,FRANK SCAFFA,NaN,NaN,NaN,NaN
4,M00431823-I1,Initial Permit,301,EAST 73 STREET,MANHATTAN,1,1044897,1448,108,NaN,...,06/04/2021 12:00:00 AM,03/11/2022 05:00:00 AM,Filing as per plans for the in,5000,BROADWAY MERCER ASSOCIATES,CYNTHIA GONZALEZ,NaN,NaN,NaN,NaN
5,B00583762-I1,Initial Permit,350,IRVING AVENUE,BROOKLYN,25,3076848,3362,304,NaN,...,09/23/2021 12:00:00 AM,11/01/2021 04:00:00 AM,NON ILLUMINATED NON ADVERTISIN,4500,NaN,YASER OBYAH,NaN,NaN,NaN,NaN
6,B00516232-I1,Initial Permit,102,PILLING STREET,BROOKLYN,29,3080348,3457,304,NaN,...,05/06/2021 12:00:00 AM,03/31/2022 12:00:00 AM,INSTALLATION OF TEMPORARY HEAV,1000,EVEREST SCAFFOLDING INC.,CHRISTOPHER DOWNES,NaN,NaN,NaN,NaN
7,B00472417-I1,Renewal Permit with Changes,1329,WILLOUGHBY AVENUE,BROOKLYN,35,3072801,3200,304,NaN,...,06/23/2021 12:00:00 AM,04/19/2022 12:00:00 AM,MODIFICATION OF PLUMBING AND S,68750,FM CONSOLIDATED HOLDING,TERESA CHUI,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241434,M00154809-I1,Initial Permit,212,WEST 22 STREET,MANHATTAN,46,1014076,771,104,NaN,...,04/26/2019 12:00:00 AM,09/30/2020 01:00:00 AM,INSTALLATION OF TEMPORARY CONS,10000,PR,STANLEY WASSERMAN,NaN,NaN,NaN,NaN
241436,Q00099914-I1,Initial Permit,259-73,148 ROAD,QUEENS,7,4290971,13687,413,NaN,...,01/31/2019 12:00:00 AM,05/06/2019 12:00:00 AM,PROPOSED TEMPORARY CONSTRUCTIO,1000,NaN,ANTHONY FINDLAY,NaN,NaN,NaN,NaN
241438,Q00001038-I1,Initial Permit,66-20,QUEENS BOULEVARD,QUEENS,19,4056084,2420,402,NaN,...,11/14/2017 12:00:00 AM,09/17/2018 12:00:00 AM,NON ILLUMINATED NON ADVERTISIN,1200,ROGOSIN INSTITUTE,AMMAR ALOW,NaN,NaN,NaN,NaN
241440,B00269752-I1,Renewal Permit Without Changes,1,MAIN STREET,BROOKLYN,7501,3346104,27,302,NaN,...,07/10/2020 12:00:00 AM,02/23/2021 12:00:00 AM,PAA being filed to reflect cha,26000,NaN,MARY FRANCES SHAUGHNESSY,NaN,NaN,NaN,NaN


Fraction of Rows with unique job numbers

In [58]:
df['Applicant License #'].nunique()/df['Applicant License #'].count()

0.03772599104329076

Group by Job # and check if latitude and longitude are the same all the same for the job, which would indicate different instances of the Job # all refer to the same Job.

In [59]:
group = df[['Applicant License #', 'Job Filing Number']].groupby('Applicant License #')

This will take a little while to run

In [60]:
tranformed = group.aggregate(lambda x: x.unique().shape[0])

Jobs with multiple latitude and longitudes:

In [61]:
potential_bad_jobs = list(tranformed)

Separate these into a temporary dataframe to play around with:

In [62]:
df_temp = df.loc[df['Applicant License #'].isin(potential_bad_jobs)].copy()

In [63]:
df_temp = df_temp.sort_values(['Job Filing Number', 'Applicant License #'])

Most of these are just missing lat and long values.

The others look to be Jobs that manage multiple houses/lots in a small area, so are probably correct

In [64]:
df_temp[df_temp.duplicated(subset=['Job Filing Number', 'Applicant License #'], keep=False)]

,Job Filing Number,Filing Reason,House No,Street Name,Borough,LOT,Bin,Block,C B NO,Apt/Condo No(s),...,Issued Date,Expired Date,Job Description,Estimated Job Costs,Owner Business Name,Owner Name,Owner Street Address,Owner City,Owner State,Owner Zip Code


#### Later, after we have cleaned more values, we will fill these missing values by job #

Remove Jobs we know to be just missing data from the list of bad jobs

In [65]:
df_temp

,Job Filing Number,Filing Reason,House No,Street Name,Borough,LOT,Bin,Block,C B NO,Apt/Condo No(s),...,Issued Date,Expired Date,Job Description,Estimated Job Costs,Owner Business Name,Owner Name,Owner Street Address,Owner City,Owner State,Owner Zip Code


Latest action date is different, so its probably differnt but very similar jobs 

Check if any Job #s have non-digit values

In [66]:
df['Applicant License #'] = df['Applicant License #'].astype('str')

In [67]:
df.loc[(~df['Applicant License #'].isna())
       &(~df['Applicant License #'].str.isdigit())]['Applicant License #']

Series([], Name: Applicant License #, dtype: object)

All Job #s entirely composed of digits, so we cast them to ints

In [68]:
df['Applicant License #'] = df['Applicant License #'].astype('str')

In [69]:
df['Applicant License #'].describe()

count     241160
unique      9098
top       032158
freq        3949
Name: Applicant License #, dtype: object

## Examining and reparing Job Filing Number

Job Filing Number appear to be mostly ints

However, there are legitimate house numbers with dashes so we'll have to make them strings

In [71]:
def show_vals(column_name, show_rows=10, df=df):
    print("Top {} {}:\n".format(show_rows, column_name))
    print(df[column_name].value_counts(dropna=False)[:show_rows])
    print()

In [72]:
show_vals('Job Filing Number', show_rows=10)

Top 10 Job Filing Number:

M00013853-I1    51
M00008120-I1    48
M00042797-I1    48
M00130581-I1    42
X00073703-I1    39
M00165956-I1    36
M00259999-I1    36
M00003926-I1    36
M00000925-I1    36
M00033650-I1    36
Name: Job Filing Number, dtype: int64



Replace NaN values with empty strings, then convert column to string, and make everything uppercase


In [73]:
df['Job Filing Number'].fillna('', inplace=True)
df['Job Filing Number'] = df['Job Filing Number'].astype('str')
df['Job Filing Number'] = df['Job Filing Number'].str.upper()

Check for numbers spelled out as words

In [74]:
df.loc[(~df['Job Filing Number'].isna())
       &(df['Job Filing Number'].str.isalpha())]['Job Filing Number']

Series([], Name: Job Filing Number, dtype: object)

Check if thses are empty strings:

In [75]:
df.loc[(~df['Job Filing Number'].str.contains('\\d', regex=True))]['Job Filing Number']

148257    PERMIT IS NO
178649    PERMIT IS NO
181411    PERMIT IS NO
Name: Job Filing Number, dtype: object

Replace spelling of numbers with their value, and remove values 'PIER',  'MANHATTAN',  'NO NUMBER'

In [102]:
df.loc[df['Job Filing Number'].str.strip('')=='PERMIT IS NO', 'Job Filing Number'] = ''


Most of these will probably be legitimate house numbers, since house numbers can have dashes

In [86]:
df.loc[(~df['Job Filing Number'].isna())
       &(~df['Job Filing Number'].str.isdigit())]['Job Filing Number']

0         M00531234-I1
1         X00496654-I1
2         B00579209-I1
3         M00514656-I1
4         M00431823-I1
              ...     
241440    B00269752-I1
241441    B00239369-I1
241442    M00083385-I1
241443    M00052214-I1
241444    B00275776-I1
Name: Job Filing Number, Length: 241160, dtype: object

Check non-numeric house #'s that don't have dashes

In [87]:
df.loc[(~df['Job Filing Number'].isna())
       &(~df['Job Filing Number'].str.isdigit())
      &(~df['Job Filing Number'].str.contains('-', regex=False))]['Job Filing Number'][:25]

148257    PERMIT IS NO
178649    PERMIT IS NO
181411    PERMIT IS NO
Name: Job Filing Number, dtype: object

We see a mix of reference to the house's garage, the rear house and single letters that likely indicate apartments in multi-occupancy venues. 

We will standardize the formatting, and maintain the reference to garage, rear, and appartment, since there is no apartment column for the job.

First split the numbers and words with a space

In [90]:
df['Job Filing Number'] = df['Job Filing Number'].str.replace(pat='(?P<one>\\d)(?P<two>[A-Z]+)', repl='\g<one> \g<two>', regex=True)

Now we will fix the formatting for garage and 
remove references to north, south, east, west, since they should be in street #

In [91]:
df['Job Filing Number'] = df['Job Filing Number'].str.replace(pat='(?P<one>GAR$)', repl='GARAGE', regex=True)

In [95]:
df['Job Filing Number'] = df['Job Filing Number'].str.replace(pat='NORTH([A-Z]+)?', repl='', regex=True)
df['Job Filing Number'] = df['Job Filing Number'].str.replace(pat='EAST([A-Z]+)?', repl='', regex=True)
df['Job Filing Number'] = df['Job Filing Number'].str.replace(pat='SOUTH([A-Z]+)?', repl='', regex=True)
df['Job Filing Number'] = df['Job Filing Number'].str.replace(pat='WEST([A-Z]+)?', repl='', regex=True)
df['Job Filing Number'] = df['Job Filing Number'].str.replace('PERMIT IS NO', repl='', regex=True)

In [96]:
# Confirm that it worked correctly:
df.loc[(~df['Job Filing Number'].isna())
       &(~df['Job Filing Number'].str.isdigit())
       &(~df['Job Filing Number'].str.contains('-', regex=False))]['Job Filing Number'][:30]

148257    
178649    
181411    
Name: Job Filing Number, dtype: object

## Examining and reparing House No

In [97]:
show_vals('House No', show_rows=10)

Top 10 House No:

1      2202
200    1306
10     1259
150    1042
55     1027
50     1020
100     960
40      871
60      861
15      853
Name: House No, dtype: int64



Replace NaN values with empty strings, then convert column to string, and make everything uppercase


In [98]:
df['House No'].fillna('', inplace=True)
df['House No'] = df['House No'].astype('str')
df['House No'] = df['House No'].str.upper()

Check for numbers spelled out as words

In [99]:
df.loc[(~df['House No'].isna())
       &(df['House No'].str.isalpha())]['House No']

13451     ONE
25394     ONE
25682     ONE
27946     ONE
30149     ONE
57475       I
58527     ONE
64326     ONE
78698     ONE
78809     ONE
79113     ONE
80899     ONE
80957     ONE
81001     ONE
102699    ONE
157256    ONE
160480    ONE
184336    ONE
195205    ONE
198739    ONE
201748      I
207613    ONE
215704    ONE
230633    ONE
Name: House No, dtype: object

Check if thses are empty strings:

In [100]:
df.loc[(~df['House No'].str.contains('\\d', regex=True))]['House No']

13451     ONE
25394     ONE
25682     ONE
27946     ONE
30149     ONE
57475       I
58527     ONE
64326     ONE
78698     ONE
78809     ONE
79113     ONE
80899     ONE
80957     ONE
81001     ONE
102699    ONE
157256    ONE
160480    ONE
184336    ONE
195205    ONE
198739    ONE
201748      I
207613    ONE
215704    ONE
230633    ONE
Name: House No, dtype: object

Replace spelling of numbers with their value, and remove values 'PIER',  'MANHATTAN',  'NO NUMBER'

In [103]:
df.loc[df['House No'].str.strip('')=='ONE', 'House No'] = ''
df.loc[df['House No'].str.strip('')=='I', 'House No'] = ''

Most of these will probably be legitimate house numbers, since house numbers can have dashes

In [104]:
df.loc[(~df['House No'].isna())
       &(~df['House No'].str.isdigit())]['House No']

12         57-28
30        112-27
35         99-06
42        133-43
43         82-08
           ...  
241425     34-19
241426     74-16
241432    161-01
241436    259-73
241438     66-20
Name: House No, Length: 40083, dtype: object

Check non-numeric house #'s that don't have dashes

In [105]:
df.loc[(~df['House No'].isna())
       &(~df['House No'].str.isdigit())
      &(~df['House No'].str.contains('-', regex=False))]['House No'][:25]

344          60B
971        111 A
1180       37GAR
1212        318A
1361       37GAR
1718       37GAR
1985        256A
2058        687A
2219    1668 GAR
2417        872A
2659    1568 AIR
3108     260 GAR
3307        500A
3360    435 REAR
3517        410A
3565    435 REAR
3645      126GAR
4881     108 1/2
5045     649 GAR
5129    435 REAR
5141        140B
5196       127 A
5605     108 1/2
5656        687A
5772        546A
Name: House No, dtype: object

We see a mix of reference to the house's garage, the rear house and single letters that likely indicate apartments in multi-occupancy venues. 

We will standardize the formatting, and maintain the reference to garage, rear, and appartment, since there is no apartment column for the job.

First split the numbers and words with a space

In [106]:
df['House No'] = df['House No'].str.replace(pat='(?P<one>\\d)(?P<two>[A-Z]+)', repl='\g<one> \g<two>', regex=True)

Now we will fix the formatting for garage and 
remove references to north, south, east, west, since they should be in street #

In [91]:
df['House No'] = df['House No'].str.replace(pat='(?P<one>GAR$)', repl='GARAGE', regex=True)

In [107]:
df['House No'] = df['House No'].str.replace(pat='NORTH([A-Z]+)?', repl='', regex=True)
df['House No'] = df['House No'].str.replace(pat='EAST([A-Z]+)?', repl='', regex=True)
df['House No'] = df['House No'].str.replace(pat='SOUTH([A-Z]+)?', repl='', regex=True)
df['House No'] = df['House No'].str.replace(pat='WEST([A-Z]+)?', repl='', regex=True)
df['House No'] = df['House No'].str.replace('PERMIT IS NO', repl='', regex=True)

In [108]:
# Confirm that it worked correctly:
df.loc[(~df['House No'].isna())
       &(~df['House No'].str.isdigit())
       &(~df['House No'].str.contains('-', regex=False))]['Job Filing Number'][:30]

344     M00502410-I1
971     M00586909-I1
1180    S00481244-I1
1212    B00552927-I1
1361    S00481244-I1
1718    S00481244-I1
1985    B00466719-I1
2058    B00345683-I1
2219    B00510739-I1
2417    B00285820-I1
2659    M00342682-I1
3108    S00308390-I1
3307    B00315901-I1
3360    M71340105-I1
3517    B00502812-I1
3565    M00370497-I1
3645    S00510751-I1
4881    M00367069-I1
5045    B00455946-I1
5129    M00370497-I1
5141    B00480587-I1
5196    B00519943-I1
5605    M00367093-I1
5656    B00500777-I1
5772    M00510354-I1
5775    Q00418044-I1
5931    X00067946-I1
5987    B00498189-I1
6071    S00078639-I1
6103    S00515398-I1
Name: Job Filing Number, dtype: object

## Checking owner's information

In [125]:
def show_vals(column_name, show_rows=10, df=df):
    print("Top {} {}:\n".format(show_rows, column_name))
    print(df[column_name].value_counts(dropna=False))
    print()

In [129]:
show_vals("Owner Business Name")
show_vals("Owner Street Address")
show_vals("Owner City")
show_vals("Owner State")
show_vals("Owner Zip Code")

Top 10 Owner Business Name:

PR                               42463
NaN                              14852
NYCHA                             5896
SPRING SCAFFOLDING LLC            3496
ROCK GROUP NY CORP                3215
                                 ...  
DARTS CONTRACTING CO                 1
23RD AVENUE REALTY                   1
NYCB PERMITS                         1
88-19 193RD STREET REALTY LLC        1
UNITED REALTY CORP                   1
Name: Owner Business Name, Length: 17521, dtype: int64

Top 10 Owner Street Address:

NaN    241160
Name: Owner Street Address, dtype: int64

Top 10 Owner City:

NaN    241160
Name: Owner City, dtype: int64

Top 10 Owner State:

NaN    241160
Name: Owner State, dtype: int64

Top 10 Owner Zip Code:

NaN    241160
Name: Owner Zip Code, dtype: int64



## Fixing owner's informations

In [131]:
df.loc[~df["Owner Business Name"].isna() & df["Owner Business Name"].str.contains("(?i)new york city")]["Owner Business Name"].value_counts()

NEW YORK CITY HOUSING AUTHORITY        841
NEW YORK CITY ECONOMIC DEVELOPMENT     195
NEW YORK CITY DEPARTMENT OF ENVIRON     41
NEW YORK CITY DEPT. OF ENVIRONMENTA     20
NEW YORK CITY DEPARTMENT OF DESIGN       7
THE HOTEL @ NEW YORK CITY                5
NEW YORK CITY TRANSIT                    5
Name: Owner Business Name, dtype: int64

Normalizes a couple of duplicate names

In [133]:
df["Owner Business Name"] = df["Owner Business Name"].str.replace("NEW YORK CITY", "NYC")
df["Owner Business Name"] = df["Owner Business Name"].str.upper()
df["Owner Business Name"] = df["Owner Business Name"].str.replace(".", '', regex=False)
df["Owner Business Name"] = df["Owner Business Name"].str.replace(",", '', regex=False)

All these are the same thing. Uses clusters to fix

In [134]:
#may have to use fuzzy/cluster to fix this problem
df.loc[~df["Owner Business Name"].isna() & df["Owner Business Name"].str.contains("(?i)HOUSING AUTHORITY")]["Owner Business Name"].value_counts()

NYC HOUSING AUTHORITY    851
Name: Owner Business Name, dtype: int64

All these states are valid

In [135]:
df["Owner State"].value_counts()

Series([], Name: Owner State, dtype: int64)

Since the states can be outside NYC, these are probably fine

In [136]:
df["Owner Zip Code"].value_counts()

Series([], Name: Owner Zip Code, dtype: int64)

## Looking at Phone Numbers:

In [137]:
show_vals("Owner Street Address")

Top 10 Owner Street Address:

NaN    241160
Name: Owner Street Address, dtype: int64



In [138]:
show_vals("Owner City")

Top 10 Owner City:

NaN    241160
Name: Owner City, dtype: int64



A lot of the nan numbers

# Data Profilling for datetime columns


Find format problems and outliers in all datetime columns

Using openclean's sklearn modules to detect problems and outliers

In [139]:
from openclean.profiling.anomalies.sklearn import DBSCANOutliers

def findDateOutliers(column_name, eps_setting = 0.05):
    datetime_data = ds.distinct(column_name)
    print("Column: ",column_name)
    
    for rank, val in enumerate(datetime_data.most_common(10)):        
        st, freq = val
        print('{:<3} {:>8}  {:>10}'.format('{}.'.format(rank + 1), st, '{:,}'.format(freq)))

    print('\nTotal number of distinct values in {} is {}'.format(column_name, len(datetime_data)))
    print(DBSCANOutliers().find(datetime_data))
    print(DBSCANOutliers(eps = eps_setting).find(datetime_data))
    print('\n==================================')

In [140]:
date_cols = []

print("Datetime Data columns:\n")
for col in ds.columns:
    if 'Date' in col or 'DATE' in col:
        print(col)
        date_cols.append(col)

print("----------------------------\n")        
        
for col in date_cols:
    findDateOutliers(col, 0.02)

Datetime Data columns:

Approved Date
Issued Date
Expired Date
----------------------------

Column:  Approved Date
1.  01/21/2020 12:00:00 AM         515
2.  02/18/2020 12:00:00 AM         515
3.  05/27/2021 12:00:00 AM         495
4.  02/25/2020 12:00:00 AM         494
5.  07/09/2021 12:00:00 AM         491
6.  11/08/2019 12:00:00 AM         482
7.  04/13/2021 12:00:00 AM         461
8.  04/21/2021 12:00:00 AM         460
9.  01/24/2020 12:00:00 AM         460
10. 03/03/2020 12:00:00 AM         457

Total number of distinct values in Approved Date is 5397
['']
['', '08/07/2019 12:00:00 AM', '02/12/2020 12:00:00 AM', '04/21/2021 12:00:00 AM', '10/12/2021 12:00:00 AM', '08/15/2019 12:00:00 AM', '11/08/2019 12:00:00 AM', '10/01/2021 12:00:00 AM', '01/21/2020 12:00:00 AM', '11/22/2020 12:00:00 AM', '05/27/2021 12:00:00 AM', '11/22/2021 12:00:00 AM', '07/09/2021 12:00:00 AM', '08/27/2019 12:00:00 AM', '12/21/2020 12:00:00 AM', '10/20/2020 12:00:00 AM', '01/12/2021 12:00:00 AM', '03/27/201

## Fixing Datetime columns format

In [142]:
datetime_column_list = date_cols
for col in datetime_column_list:
    show_vals(col)

Top 10 Approved Date:

01/21/2020 12:00:00 AM    515
02/18/2020 12:00:00 AM    512
02/25/2020 12:00:00 AM    494
05/27/2021 12:00:00 AM    494
07/09/2021 12:00:00 AM    491
                         ... 
09/26/2018 08:45:00 AM      1
11/03/2017 01:53:00 PM      1
01/09/2019 10:55:00 AM      1
01/12/2018 12:17:00 PM      1
04/26/2019 10:22:00 AM      1
Name: Approved Date, Length: 5397, dtype: int64

Top 10 Issued Date:

11/15/2021 12:00:00 AM    713
11/04/2021 12:00:00 AM    679
05/10/2021 12:00:00 AM    671
09/29/2021 12:00:00 AM    662
04/15/2021 12:00:00 AM    658
                         ... 
03/16/2017 12:00:00 AM      1
08/21/2021 12:00:00 AM      1
09/13/2020 12:00:00 AM      1
01/26/2019 12:00:00 AM      1
06/22/2019 12:00:00 AM      1
Name: Issued Date, Length: 1254, dtype: int64

Top 10 Expired Date:

08/06/2020 12:00:00 AM    6213
12/31/2021 05:00:00 AM    5618
01/31/2021 12:00:00 AM    3708
12/31/2020 12:00:00 AM    3567
01/01/2022 05:00:00 AM    2874
                       

Check to see if any columns have values in year-month-day format

In [143]:
for col in datetime_column_list:
    print(col, '\n', df.loc[df[col].str.contains('-', regex=False, na=False)][col], '\n\n')

Approved Date 
 Series([], Name: Approved Date, dtype: object) 


Issued Date 
 Series([], Name: Issued Date, dtype: object) 


Expired Date 
 Series([], Name: Expired Date, dtype: object) 




### Check the coherence of datetime values

These don't make sense, but it's not entirely clear if they should be swapped, or removed or what

In [151]:
df.loc[(df['Approved Date'] < df['Expired Date'])
      &(~df['Approved Date'].isna() & ~df['Expired Date'].isna())][['Approved Date', 'Expired Date']]

,Approved Date,Expired Date
0,06/23/2021 12:00:00 AM,06/23/2022 12:00:00 AM
1,05/06/2021 12:00:00 AM,12/31/2021 12:00:00 AM
5,09/20/2021 12:00:00 AM,11/01/2021 04:00:00 AM
7,02/24/2021 12:00:00 AM,04/19/2022 12:00:00 AM
11,08/10/2020 12:00:00 AM,12/26/2021 12:00:00 AM
...,...,...
241439,10/03/2018 12:00:00 AM,10/03/2019 12:00:00 AM
241441,06/29/2020 12:00:00 AM,07/20/2021 12:00:00 AM
241442,10/03/2018 12:00:00 AM,11/03/2020 12:00:00 AM
241443,07/06/2018 12:00:00 AM,09/30/2018 12:00:00 AM


In [152]:
df.loc[(df['Approved Date'] > df['Expired Date'])
      &(~df['Approved Date'].isna() & ~df['Approved Date'].isna())][['Approved Date', 'Expired Date']]

,Approved Date,Expired Date
2,08/19/2021 12:00:00 AM,07/20/2022 12:00:00 AM
3,05/06/2021 12:00:00 AM,04/01/2022 12:00:00 AM
4,12/14/2020 12:00:00 AM,03/11/2022 05:00:00 AM
6,05/06/2021 12:00:00 AM,03/31/2022 12:00:00 AM
8,05/18/2020 12:00:00 AM,02/25/2021 12:00:00 AM
...,...,...
241428,09/05/2019 12:00:00 AM,08/18/2020 12:00:00 AM
241436,12/18/2018 12:00:00 AM,05/06/2019 12:00:00 AM
241437,09/27/2019 12:00:00 AM,02/28/2021 12:00:00 AM
241438,11/14/2017 12:00:00 AM,09/17/2018 12:00:00 AM


Again here it's not clear how a job could be approved before being assigned, or how a job could have it's latest action date  before the Pre-filing date 

In [153]:
df.loc[(df['Issued Date'] > df['Approved Date'])
      &(~df['Issued Date'].isna() & ~df['Approved Date'].isna())][['Issued Date', 'Approved Date']]

,Issued Date,Approved Date
5,09/23/2021 12:00:00 AM,09/20/2021 12:00:00 AM
7,06/23/2021 12:00:00 AM,02/24/2021 12:00:00 AM
12,05/06/2021 12:00:00 AM,01/07/2020 12:00:00 AM
14,11/16/2020 12:00:00 AM,11/13/2020 12:00:00 AM
17,08/19/2021 12:00:00 AM,06/29/2018 12:00:00 AM
...,...,...
241434,04/26/2019 12:00:00 AM,04/24/2019 12:00:00 AM
241435,11/07/2018 12:00:00 AM,08/25/2018 12:00:00 AM
241437,09/30/2020 12:00:00 AM,09/27/2019 12:00:00 AM
241440,07/10/2020 12:00:00 AM,03/04/2020 12:00:00 AM


# Data Profilling for City and Other Description

Find format problems and outliers in City and Description columns

Using openclean's sklearn modules to detect problems and outliers

In [154]:
from openclean.profiling.anomalies.sklearn import DBSCANOutliers

# Print the ten most frequent values for the 'Vehicle Expiration Date' column.
def findDateOutliers(column_name, eps_setting = 0.05):
    applicant_data = ds.distinct(column_name)
    print("Column: ",column_name)
    
    for rank, val in enumerate(applicant_data.most_common(10)):        
        st, freq = val
        print('{:<3} {:>8}  {:>10}'.format('{}.'.format(rank + 1), st, '{:,}'.format(freq)))

    print('\nTotal number of distinct values in {} is {}'.format(column_name, len(applicant_data)))
    print(DBSCANOutliers(eps = eps_setting).find(applicant_data))
    print('\n==================================')

In [155]:
date_cols = ["Borough", "Street Name"]

In [156]:

print("----------------------------\n")        
        
for col in date_cols:
    findDateOutliers(col, 0.1)

----------------------------

Column:  Borough
1.  MANHATTAN     104,181
2.  BROOKLYN      62,626
3.    QUEENS      41,561
4.     BRONX      22,068
5.  STATEN ISLAND      11,009

Total number of distinct values in Borough is 5
['QUEENS', 'STATEN ISLAND', 'BROOKLYN', 'BRONX', 'MANHATTAN']

Column:  Street Name
1.  BROADWAY       6,902
2.  PARK AVENUE       4,056
3.  5 AVENUE       3,543
4.  MADISON AVENUE       2,942
5.  3 AVENUE       2,083
6.  LEXINGTON AVENUE       1,984
7.  FIFTH AVENUE       1,649
8.  7 AVENUE       1,465
9.  AVENUE OF THE AMERICAS       1,343
10. 6 AVENUE       1,333

Total number of distinct values in Street Name is 5601
['BR 6 STREET', 'BRIGHTON   10 LANE', 'BAYSIDE', 'PPW', 'AVENUE OF THE AMERICAS', 'BOWERY', 'PIER   76', 'BRIGHTON 1 PLACE', 'BRIGHTON 2 PLACE', 'PIER   57', 'LEXINGTON AVENUE', 'CPW', 'MADISON AVENUE', 'BRIGHTON 10 LANE', 'LITTLE WEST   12 STREET', 'RIVERSIDE DRIVE', 'INTERNATIONAL AIRPORT CENTER BL', 'BQE', 'BRIGHTON 5 PLACE', '5 AVENUE', 'BROA

In [157]:
from openclean.function.eval.base import Col, Eval
from openclean.function.eval.logic import And
from openclean.function.value.phonetic import Soundex, soundex

In [158]:
upper = ds\
    .select('Borough')\
    .update('Borough', str.upper)

In [159]:
from openclean.data.refdata import RefStore

refdata = RefStore()
city_df = refdata\
    .load('encyclopaedia_britannica:us_cities', auto_download=True)\
    .df()


In [160]:
city_list = city_df['city']
print(city_list)

0          Demopolis
1          Sylacauga
2               Troy
3             Dothan
4           Prichard
            ...     
1956          Powell
1957        Riverton
1958        Sheridan
1959    Rock Springs
1960         Buffalo
Name: city, Length: 1961, dtype: object


# An example of using soundex in openclean

However, using soundex for each of the city is too slow, the code below take nearly 4 mins for one sningle city.\
So we should use clustering first and then use hard code to clean the remianing city name that is not in the city_list.

In [162]:
brooklyn = ds\
    .select('Borough')\
    .update('Borough', str.upper)\
    .filter(And(Eval('Borough', Soundex()) == soundex('BROOKLYN'), Col('Borough') != 'BROOKLYN'))\
    .distinct()

print('RANK\tCOUNT\tNAME')
for i, entry in enumerate(brooklyn.most_common()):
    key, count = entry
    print('{}.\t{}\t{}'.format(i + 1, count, key))

RANK	COUNT	NAME


# Data Cleaning for Applicant columns

* how to deal with empty values has not decided yet

# Check State Column

In [163]:
state_col = 'Owner State'
findDateOutliers(state_col, 0.1)

Column:  Owner State
1.               241,445

Total number of distinct values in Owner State is 1
['']



# Apply similar operation on Owner's Business Name

In [164]:
bn_col = "Owner Business Name"
findDateOutliers(bn_col)

Column:  Owner Business Name
1.        PR      42,491
2.                10,551
3.     NYCHA       5,929
4.       N/A       3,635
5.  SPRING SCAFFOLDING LLC       3,499
6.  ROCK GROUP NY CORP       3,223
7.  OWNERS REP       2,369
8.  PHOENIX SUTTON STR INC       2,237
9.  TRI BOROUGH SCAFFOLDING       1,939
10. SAFWAY ATLANTIC LLC       1,861

Total number of distinct values in Owner Business Name is 17524
['', 'SEQUOIA@RE-MGT.COM', 'ST. PAUL CHONGHASANG R.C.C.', '731 M & M DELI GROCERY', '221 W 116, LLC', '30494 REALTY CORP', 'M.J. MARTIN & SON INC.', '157 West 24h Street Lodging LLC', '111-115 NORTH 5TH STREET LLC', 'RICHTER+RATNERCONTRACTING CORP.', 'RFR/SF 17 STATE STREET, L.P.', 'FRICK AND FRACK 2 LLC', 'M.S.T. RANDALL, LLC', 'A&J 19041 111TH ROAD  MANAGMENT LLC', '273 PROPERTIES', '9TH AVE BAKERY', '117-119 LEASING CORP.', '1624 E 12TH ST INC.', 'EASTCHESTER-ASTOR, LLC', 'LTCELECTRIC, INC.', '718-484-7861', 'RIVERATONYCONSTRUCTON.INC', 'HDFC611', '- NONE -', '23-81 21 LAC LLC', '

# Using clustering for Business Name takes too much time, we can only clean those empty data for now 

In [165]:
df[bn_col] = df[bn_col].replace(['N/A', '', 'NA','NONE'], [None,None,None,None])

# Data Profilling for applicant columns

Find format problems and outliers in all applicant columns

Using openclean's sklearn modules to detect problems and outliers

In [166]:
from openclean.profiling.anomalies.sklearn import DBSCANOutliers

# Print the ten most frequent values for the 'Vehicle Expiration Date' column.
def findDateOutliers(column_name, eps_setting = 0.05):
    applicant_data = ds.distinct(column_name)
    print("Column: ",column_name)
    
    for rank, val in enumerate(applicant_data.most_common(10)):        
        st, freq = val
        print('{:<3} {:>8}  {:>10}'.format('{}.'.format(rank + 1), st, '{:,}'.format(freq)))

    print('\nTotal number of distinct values in {} is {}'.format(column_name, len(applicant_data)))
    print(DBSCANOutliers(eps = eps_setting).find(applicant_data))
    print('\n==================================')

In [167]:
date_cols = []

print("Applicant Data columns:\n")
for col in ds.columns:
    if 'Applicant' in col:
        print(col)
        date_cols.append(col)

Applicant Data columns:

Applicant License #
Applicant First Name
Applicant Middle Name
Applicant Last Name
Applicant Business Name
Applicant Business Address


In [168]:
date_cols = []

print("Applicant Data columns:\n")
for col in ds.columns:
    if 'Applicant' in col:
        print(col)
        date_cols.append(col)

print("----------------------------\n")        
        
for col in date_cols:
    findDateOutliers(col, 0.1)

Applicant Data columns:

Applicant License #
Applicant First Name
Applicant Middle Name
Applicant Last Name
Applicant Business Name
Applicant Business Address
----------------------------

Column:  Applicant License #
1.    032158       3,950
2.    607447       3,506
3.    610440       3,329
4.    617232       2,374
5.    606141       2,371
6.    609527       2,225
7.    616234       2,103
8.    018914       2,008
9.    608211       1,989
10.   620254       1,786

Total number of distinct values in Applicant License # is 9098
['607447', '555555', '616836', '999999', '610440', '618253', '609527', '602375', '606141', '032158']

Column:  Applicant First Name
1.   MICHAEL       7,879
2.   WILLIAM       6,720
3.    JOSEPH       6,034
4.  CHRISTOPHER       5,212
5.      JOHN       5,202
6.    ROBERT       4,519
7.  SIMRANPAL       3,326
8.    THOMAS       3,102
9.     JAMES       2,907
10.    FRANK       2,777

Total number of distinct values in Applicant First Name is 3394
['SENIA DE DIOS',

# Analysis

the above results show the problems for the data cleaning task:
    
### For name data

in "Applicant's First Name", "Applicant's Last Name", "Applicant Professional Title", there are many outliers which are illegal input, and there are many similar values. We need first converts evident outliers to legal values, then use kNN clusterer to standardize similar values.


### Applicant License #

Applicant License # is made of 6 digits, there are outliers that do not satisfy the 6-digit format. We can not use kNN clusterer to standardize because many License # are similar. 

# Data Cleaning for Applicant columns

* how to deal with empty values has not decided yet

In [171]:
# mapping list to replace outliers
outlier1 = ['', 'MR. ROSS ADAM C', 'MICHAEL', 'N. J.', 'WILLIAM 11', 'JOSEP;H``', 'DAID/11/2007', 'CHUNG   LUN', '718 9215010', 'ANTHONY', 'HSIA0-NAN', 'JOSEPH', '``````````', 'ROBERT  `', 'RAJENDRA9956700', '2', 'G.B.M.', 'EUGENE......JR', '6312100', 'CLAUDE,JR.', 'THOMAS``', 'ALAN  L', 'Nab53', 'MR. Y. B', 'J.J', 'PH8ILIP', 'I. M', 'RICHARD', 'ALBERTA S 111 D', 'P ;', 'GENECG.C. ENG &', 'J.J.', '2126202794', 'SHAW  HWA', 'HARRY         H', 'MR DOU8GLAS', '`1D', 'PAUL', 'K. T.', 'JOHN', '...NORMAN', 'EVAN   D', '7184361278BERNA', 'S.D. DON', 'KY00 SUK', 'JJ', 'YURI.`', 'MAD/Y/ARNI', 'ES ON SCH B', 'EUGENE.......JR', 'NEAL', 'F._ERIC', 'RYAN,  JR', 'AASDFASDFASDF', 'LA0-TECH', 'RODNEY   __', 'DAVID', 'G. L.', 'JAMES', 'LESLI8E', '7186054055', 'GEORGE', 'G.B.M', 'DAVID    JON', 'CHUNG---YAO', 'PETER', 'YUBUN(JACK)', 'GLEN A. L.', '1P', 'JUDE.....N.O', 'LEONARD--', 'WILLIAM', 'ANTHONY,111', 'WU(WOODY)', 'GAD/HON-AN', 'GLEN  A.L.', 'J.B. Jr.', 'LORENZO..A', 'J J', '..RAMSEY', 'HUI LI I', 'ANTONIO9', 'ROBERT', '0.BERT', 'DUMMY 2', '...JOSEPH', 'RUSSELL 111', 'THOMAS', 'H./E./CAMELLE', 'LALAL', 'M.E. P.E', 'R0OBIN VINCENT', '--young', 'AKM', 'LE1', 'IK.T.', 'LEO, JR.', 'J. Butch A. Jr.', 'WU (WOODY0', 'PAUL   N', 'CHRISTOPHER']
mapping1 = [None, 'ROSS ADAM C', 'MICHAEL', 'N. J.', 'WILLIAM', 'JOSEPH', None, 'CHUNG LUN', None, 'ANTHONY', 'HSIA0 NAN', 'JOSEPH', None, 'ROBERT', 'RAJENDRA', None, 'G.B.M.', 'EUGENEJR', None, 'CLAUDE JR.', 'THOMAS', 'ALAN  L', 'Nab', 'MR. Y. B', 'J.J', 'PHILIP', 'I. M', 'RICHARD', 'ALBERTA', None, 'GENECG.C. ENG', 'J.J.', None, 'SHAW HWA', 'HARRYH', 'MR DOUGLAS', None, 'PAUL', 'K. T.', 'JOHN', 'NORMAN', 'EVAND', 'BERNA', 'S.D. DON', 'KY00 SUK', 'JJ', 'YURI.`', 'MADYARNI', 'ES ON SCH B', 'EUGENEJR', 'NEAL', 'FERIC', 'RYAN,  JR', 'AASDFASDFASDF', 'LA0 TECH', 'RODNEY', 'DAVID', 'G. L.', 'JAMES', 'LESLIE', None, 'GEORGE', 'G.B.M', 'DAVID JON', 'CHUNG YAO', 'PETER', 'YUBUN(JACK)', 'GLEN A. L.', None, 'JUDE N.O', 'LEONARD--', 'WILLIAM', 'ANTHONY,111', 'WU(WOODY)', 'GAD HON-AN', 'GLEN A.L.', 'J.B. Jr.', 'LORENZOA', 'J J', 'RAMSEY', 'HUI LI I', 'ANTONIO9', 'ROBERT', '0.BERT', 'DUMMY', 'JOSEPH', 'RUSSELL', 'THOMAS', 'H.E.CAMELLE', 'LALAL', 'M.E. P.E', 'R0OBIN VINCENT', 'young', 'AKM', 'LE1', 'IK.T.', 'LEO, JR.', 'J. Butch A. Jr.', 'WU (WOODY0', 'PAUL   N', 'CHRISTOPHER']

outlier2 = ['SHARMA #0', "0'CONNOR", 'RUSHTON    UEL', 'UDDIN   Z', 'HINKLEY 1', 'O&#039;CONNOR, P.E.', '.OOK', 'SAMUELS111', 'O&#039;CONNOR', 'CALIENDO', 'SMITH   JR.', 'LO  BUE', '7AN', '+-+ETTIERI', 'SMITH, 111', 'KAMEN   1', '.EE', 'MASS, 1', '.EI', 'Zagaroli 3rd', 'RINI   II', 'KAMEN   R', 'RYAN 11', 'SPI8EZIA L S', 'MUFTIC..A.I.A', 'COSTELLO9 RA A I A', 'CALVANICO', 'LLC.', 'POEPPEL, P.E.', 'HAMA07', 'HINLEY,1', '1212', "O  ' CONNELL", 'HURT,JR.,', 'WESOLOWSKI', 'CHEN', '`ING, R.A', 'MARTARELLA 111', 'Gandhi, Ph.D., P.E.', '90I', 'ENNIS 2', 'COSTELLO R A A I A', '3UI', 'N/A', 'HURT,  JR', 'LEHR,1', 'KOHLER, 111', 'GERAZOUNIS', 'Alexander,1', 'LUBOW, R.A. LEED AP', 'RINI,111', '08CZAK', '````````````````````', 'CHAO  R.A.', 'Geier 11', '08NGEL', '08SOLOWSKI', 'I11', 'HINKLEY, 1', 'RUDIKOFF, P.E.', "O'CONNOR", 'SHAH   EZ', 'MIELE, JR., P.E.', 'RITTENHOUSE 111', 'AMADI   ISIOFIA', 'HINKLEY,1', 'RENFORE````````', "O'HARA,JR.", '73020012', 'PHAGOO   I', 'BRAY.....,', 'LLL', 'BHATHIA,1', 'GANDHI, PH. D., P.E', 'KO K', 'VASSALOTTI 11', 'HURT, JR .', '0018LKLE', 'RINI -111', 'PARIHAR', 'EE', 'L00802', 'ELISE.111', 'KING , R.A', 'CHRYSLER  P E', 'LEHR 1', 'Walters   Jr.', 'LEE', 'RINI  III', 'D&#039;ANGELO', '0UDOLPH III', 'VIEHE-NAESS 111', ',MO', '08E', '47DIKOFF', 'Yu,', '420865380', 'COPELAND', 'ZWIEFEL 3RD', 'PETERSEN', 'King, R.A.,', 'RINI, III', '7APA', 'CHEN   S', 'Hurt  Jr.', 'KATZ', 'NIZAMBAD.(P.E.)', '901BEN', '4153LOO', 'SYED-NAQVI', 'RYAN , JR.', 'K O K O R I S', 'ELISEO111', 'O&#039;CONNELL', 'ZEID61', '---Lewis', '00CHELI', 'MOHAMMAD       +++++', 'METZLER  P E', 'BAILEY', 'GANDHI, PH. D., P.E.', 'TIEMANN.111', 'SMITH.111', 'DI GER0NIMO', 'GANDHI, PH,D., P.E', 'III', 'J C', 'MAGAMI-QAIM-MAGAMI', '+M', 'LO G1UDICE', 'HOQUE', 'RUDIKOFF', 'Y10007OR', 'SMITH,111', 'KING R A FAIA', 'RYAN III, AIA', '08AN', 'STARK 1', 'MASS', 'VICTORI0, R.A', 'RIZVI   A', '21029677', "3'CONNOR", 'Wong /  Lai', 'KAPLAN 3', 'GRAICHEN.JR./DAWN/DI', 'GROSSMAN ,PE,F.A.C.I']
mapping2 = ['SHARMA ', "CONNOR", 'RUSHTON UEL', 'UDDIN Z', 'HINKLEY ', 'CONNOR P.E.', None, 'SAMUELS', 'CONNOR', 'CALIENDO', 'SMITH JR.', 'LO BUE', None, 'ETTIERI', 'SMITH', 'KAMEN', '.EE', 'MASS', '.EI', 'Zagaroli', 'RINI', 'KAMEN R', 'RYAN', 'SPIEZIA L S', 'MUFTIC.A.I.A', 'COSTELLO9 RA A I A', 'CALVANICO', 'LLC.', 'POEPPEL P.E.', 'HAMA', 'HINLEY', None, "CONNELL", 'HURT JR.', 'WESOLOWSKI', 'CHEN', 'ING R.A', 'MARTARELLA', 'Gandhi', None, 'ENNIS ', 'COSTELLO R A A I A', None, None, 'HUR  JR', 'LEHR', 'KOHLER 111', 'GERAZOUNIS', 'Alexander', 'LUBOW R.A. LEED AP', 'RINI',None, None, 'CHAO R.A.', 'Geier', None, 'SOLOWSKI', None, 'HINKLEY', 'RUDIKOFF, P.E.', "CONNOR", 'SHAH EZ', 'MIELE JR. P.E.', 'RITTENHOUSE', 'AMADI   ISIOFIA', 'HINKLEY', 'RENFORE', "O'HARA,JR.", None, 'PHAGOO I', 'BRAY,', 'LLL', 'BHATHIA', 'GANDHI', 'KO K', 'VASSALOTTI', 'HURT JR.',None, 'RINI', 'PARIHAR', 'EE', None, 'ELISE', 'KING R.A', 'CHRYSLER  P E', 'LEHR', 'Walters Jr.', 'LEE', 'RINI  III', 'ANGELO', '0UDOLPH III', 'VIEHE-NAESS', 'MO', '08E', None, 'Yu,', None, 'COPELAND', 'ZWIEFEL 3RD', 'PETERSEN', 'King, R.A.,', 'RINI, III', '7APA', 'CHEN   S', 'Hurt  Jr.', 'KATZ', 'NIZAMBAD.(P.E.)', None, None, None, 'RYAN JR.', 'KOKORIS', 'ELISE', 'CONNELL', None, 'Lewis', 'CHELI', 'MOHAMMAD', 'METZLER  P E', 'BAILEY', 'GANDHI', 'TIEMANN', 'SMITH', 'DI GER0NIMO', 'GANDHI', 'III', 'J C', 'MAGAMI QAIM MAGAMI', None, 'LO G1UDICE', 'HOQUE', 'RUDIKOFF', None, 'SMITH', 'KING R A FAIA', 'RYAN III AIA', None, 'STARK', 'MASS', 'VICTORI0 R.A', 'RIZVIA', None, "CONNOR", 'Wong Lai', 'KAPLAN', 'GRAICHEN.JR. DAWN DI', 'GROSSMAN']

outlier3 = ['', '....DEMO', '050069', 'DEM. CONTR.,', 'XXXXX', 'G/C 10114H9', 'CGWC10114H99', '00', 'X S000155', '082-36-1245', 'G.G', 'LESSEE', '......GC', "'", '..OWNER', 'GC 2293', '--', 'XXXXXX', 'LS 31,721', '...GC', 'gen.cont.', 'G.C TK#4592', 'PE', 'RLA - 818', '.....OWNER', 'RLA 16077', 'G C', 'X 4129892', 'G. C.', 'R.L.A', 'GC 1028350', 'WC10114H99', 'LEESEE', 'GEN.CONT.', 'SIGN..HANGER', 'DEMO 20451', 'D8615', '.X', 'P.L.L.C', '..DEMO', 'G .C', 'L A', 'G.C NY11101', '32820', '....OWNER', 'GC(DEMO)', 'C0NTRACTOR', 'EXPEDITORC99792', 'X 1341946', 'TRACK# 1390', 'EXPED.R4466', 'PLLC 9599691', 'G.C 1110101', '029649', '(CHECK)', 'DEM. CONTR,', 'EXPEDIT(H66172)', '.........GC', 'CITY OF N Y', 'GC 1170386', 'G. C', 'CO0OWNER', '(CHECKED)', 'C.C', '23392 1159774', 'DEMO {', 'RA', 'T. 31132', '....GC', 'RLA-787', 'TRACK #1390', 'D C', 'G.CONTR.', 'DEMO  CONT', '1GC', 'CC', 'demo G.C.', 'TRACK. #1390', 'M.F.S.P.C.', '...DEMO', 'DEMO G C', '13328', 'GEN  CONT', 'GC 1221073', "GC;'", 'DEMO 1341946', '11234', 'G.C.,', '.....GC', 'LIC.133668259 1', '?', '0WNER', 'C10892', 'GEN..CONT']
mapping3 = [None, 'DEMO', None, 'DEM. CONTR', None, 'G/C', 'CGWC', None, 'X S', None, 'G.G', 'LESSEE', 'GC', None, 'OWNER', 'GC', None, None, 'LS ', 'GC', 'gen.cont.', 'G.C TK', 'PE', 'RLA ', 'OWNER', 'RLA ', 'G C', 'X', 'G. C.', 'R.L.A', 'GC', 'WC', 'LEESEE', 'GEN.CONT.', 'SIGN.HANGER', 'DEMO', None,None, 'P.L.L.C', 'DEMO', 'G.C', 'L A', 'G.C ', None, 'OWNER', 'GC(DEMO)', 'C0NTRACTOR', 'EXPEDITORC', None, 'TRACK', 'EXPED.R', 'PLLC ', 'G.C', None, None, 'DEM. CONTR,', 'EXPEDIT', 'GC', None, 'GC', 'G.C', 'CO0OWNER', None, 'C.C', None, 'DEMO', 'RA', None, 'GC', 'RLA', None, 'D C', 'G.CONTR.', 'DEMO  CONT', 'GC', 'CC', 'demo G.C.', None, 'M.F.S.P.C.', 'DEMO', 'DEMO G C', None, 'GEN  CONT', 'GC ', "GC ", 'DEMO ', None, 'G.C.', 'GC', 'LIC', None, '0WNER',None, 'GEN.CONT']

outlier4 = ['', '0000GC', '083278', 'DD5615', '0000PB', '00ASB4', 'B81923', '99998', '000N/A', '65569+', '01827O', 'R9526', 'LP0256', 'N/A', '1964', 'ISLAND', '1609', '000PW1', '00DEMO', '0688.6', '00000', '.20929', 'LP0258', '000TOR', '0D8615', '0SWITA', '818', 'O02200', 'DEMO', '196', '1075', '0000NT', '215', '0', '00000`', "D'ALTO", '0455', '22377', 'DD8615', '050579', '226', 'SWITA', 'DD6815', 'X02689']
mapping4 = [None, '0000GC', '083278', 'DD5615', '0000PB', '00ASB4', 'B81923', '099998', '000000', '065569', '01827O', '0R9526', 'LP0256',None, '001964',None, '001609', '000PW1', '00DEMO', '006886', '000000', '020929', 'LP0258', '000TOR', '0D8615', '0SWITA', '000818', 'O02200', None, '000196', '001075', '0000NT', '000215', '000000', '000000', None, '000455', '022377', 'DD8615', '050579', '000226', None, 'DD6815', 'X02689']

outlier5 = ['', '0000GC', '083278', 'DD5615', '0000PB', '00ASB4', 'B81923', '99998', '000N/A', '65569+', '01827O', 'R9526', 'LP0256', 'N/A', '1964', 'ISLAND', '1609', '000PW1', '00DEMO', '0688.6', '00000', '.20929', 'LP0258', '000TOR', '0D8615', '0SWITA', '818', 'O02200', 'DEMO', '196', '1075', '0000NT', '215', '0', '00000`', "D'ALTO", '0455', '22377', 'DD8615', '050579', '226', 'SWITA', 'DD6815', 'X02689']
mapping5 = [None, '0000GC', '083278', 'DD5615', '0000PB', '00ASB4', 'B81923', '099998', '000000', '065569', '01827O', '0R9526', 'LP0256',None, '001964',None, '001609', '000PW1', '00DEMO', '006886', '000000', '020929', 'LP0258', '000TOR', '0D8615', '0SWITA', '000818', 'O02200', None, '000196', '001075', '0000NT', '000215', '000000', '000000', None, '000455', '022377', 'DD8615', '050579', '000226', None, 'DD6815', 'X02689']

outlier6 = ['','77 ALLEN STREET.....2B', '50 LEFFERTS AVE......APT2J', '90' ]
mapping6 = [None, '77 ALLEN STREET 2B', '50 LEFFERTS AVE APT2J', None]


outliers = [outlier1, outlier2, outlier3, outlier4, outlier5, outlier6]
mappings = [mapping1, mapping2, mapping3, mapping4, mapping5, mapping6]



# Remove evident outliers using hard coded mapping

In [172]:
i = 0
for col in date_cols:
    df[col] = df[col].replace(outliers[i], mappings[i])
    i += 1

# Convert similar values to suggested value using kNN clustering

In [173]:
# Cluster string using kNN clusterer (with the default n-gram setting)
# using the Levenshtein distance as the similarity measure.

from openclean.cluster.knn import knn_clusters
from openclean.function.similarity.base import SimilarityConstraint
from openclean.function.similarity.text import LevenshteinDistance
from openclean.function.value.threshold import GreaterThan

def getClusters(col, minsize = 2):
    dba = ds.select(col).distinct()
    clusters = knn_clusters(
        values=dba,
        sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.75)),
        minsize=minsize
    )
    return clusters

def print_cluster(cnumber, cluster):
    print('Cluster {} (of size {})\n'.format(cnumber + 1, len(cluster)))
    for val, count in cluster.items():
        print('{} ({})'.format(val, count))
    print('\nSuggested value: {}\n\n'.format(cluster.suggestion()))

def updateUsingClusters(col, clusters, isPrint = False):
    
    orignal_list = []
    suggestion_list = []
    clusters.sort(key=lambda c: len(c), reverse=True)
       
    for i, cluster in enumerate(clusters):        
        suggestion = cluster.suggestion()
        orignal_list = []
        suggestion_list = []
        if isPrint and i < 5:
            print_cluster(i, cluster)
        
        for val, count in cluster.items(): 
            orignal_list.append(val)
            suggestion_list.append(suggestion)
            
    df[col] = df[col].replace(orignal_list, suggestion_list)

In [174]:
for col in date_cols[:3]:
    print("kNN cluster for ", col)
    col_clusters = getClusters(col)
    print("updating column ", col)
    print("----------------------\nTop 5 Cluster:\n----------------------")
    updateUsingClusters(col, col_clusters, True)
    print("================")

kNN cluster for  Applicant License #
updating column  Applicant License #
----------------------
Top 5 Cluster:
----------------------
kNN cluster for  Applicant First Name
updating column  Applicant First Name
----------------------
Top 5 Cluster:
----------------------
Cluster 1 (of size 10)

MOHAMMAD (330)
MOHAMMAD M (4)
MOHAMMAD I (5)
MOHAMMAD O (3)
MUHAMMAD U (3)
MOHAMMAD M. (2)
MOHAMMAD J (5)
MUHAMMAD A (2)
MOHAMMED A (6)
MOHAMMAD S (8)

Suggested value: MOHAMMAD


Cluster 2 (of size 10)

HARVINDER (82)
DARVINDER (6)
PARVINDER (17)
TARVINDER (4)
RAVINDER (2)
DEVINDER (32)
KULVINDER (6)
DAVINDER (4)
ARVINDER (1)
DALVINDER (576)

Suggested value: DALVINDER


Cluster 3 (of size 10)

DALVINDER (576)
HARVINDER (82)
PARVINDER (17)
TARVINDER (4)
GURVINDER (25)
RAVINDER (2)
DEVINDER (32)
DAVINDER (4)
ARVINDER (1)
DARVINDER (6)

Suggested value: DALVINDER


Cluster 4 (of size 9)

DALVINDER (576)
DARVINDER (6)
PARVINDER (17)
TARVINDER (4)
GURVINDER (25)
RAVINDER (2)
DAVINDER (4)
ARVINDER (

# Save cleaned data

In [176]:
outputpath = datafile[2:11]+'_cleaned_data.csv'
df.to_csv(outputpath,sep=',',index=False,header=True) 